
# 跨声速抖振非定常流场预测

## 环境安装

本案例要求 **MindSpore >= 2.0.0** 版本以调用如下接口: *mindspore.jit, mindspore.jit_class, mindspore.data_sink*。具体请查看[MindSpore安装](https://www.mindspore.cn/install)。

此外，你需要安装 **MindFlow >=0.1.0** 版本。如果当前环境还没有安装，请按照下列方式选择后端和版本进行安装。

In [ ]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 概述

在跨声速流动条件下，翼型上表面出现的自持续大尺度激波振荡现象被称为跨声速抖振。其原因与激波与边界层流动分离及相互作用有关。进入抖振边界后，分离区变化引起流动不稳定，影响了激波的流场位置，使得激波产生前后运动，具有复杂的非定常和非线性特征。从流场(时空流)直接学习非定常激波抖振特征对抖振研究而言是一种有价值且具有挑战性的方法。为了找到一个高效的DL建模方法解决复杂非定常跨声速抖振问题，提出一种增强型混合深度神经网络(eHDNN)，基于流场重构对非定常流场进行预测

![p1.png](./images/p1.png)

## 模型架构

eHDNN的基本框架主要基于混合深度神经网络框架，主要由卷积神经网络（CNNs）、卷积长短时间记忆网络（ConvLSTMs）和反卷积神经网络（DeCNNs）组成。CNN降低了时间序列流场的维数，实现特征提取;ConvLSTM学习低维时空特征并进行预测;最后，DeCNN实现预测流场的重建

+ 输入层：输入历史流场
+ 卷积层：通过CNN对流场进行降维，提取高维时空流动特征
+ 记忆层：通过ConvLSTM学习低维空间流场时空特征的演变，预测下一时刻
+ 反卷积输出层：将预测的流场低维特征恢复到高维空间，通过DeCNN实现对下一时刻瞬态流场预测，并输出预测结果

![p2.jpg](./images/p2.jpg)

## 训练数据集

由OAT15A超临界翼型非定常抖振的数值仿真流场数据构建的多维矩阵流场快照矩阵构建而成。分为单状态数据集和多状态数据集两种

+ 多状态数据集为多攻角状态下的非定常抖振流场序列数据，攻角状态包括3.3°、3.4°、3.5°、3.6°、3.7°、3.8°六个攻角，均位于抖振边界内
+ 单状态数据集为单一攻角状态下的非定常抖振流场序列数据，攻角状态为上述攻角的任意一个（默认3.5°）
+ 每张流场快照包含3个通道，代表流场的压强分布信息、弦向速度信息、法向速度信息，多维矩阵流场快照矩阵尺寸为：T×C×H×W(C=3,H=200,W=200,C为通道数，H，W分别为快照的高和宽）

In [1]:
import os
import time
import argparse
import numpy as np

from mindspore import nn, ops, context, save_checkpoint, jit, data_sink, set_seed
from mindspore.amp import DynamicLossScaler, auto_mixed_precision, all_finite
from mindflow.utils import load_yaml_config

In [2]:
from src import create_dataset, ForwardNet, HybridLoss

## 训练环境

+ 训练采用Mindspore框架的静态图模式（GRAPH）
+ 在GPU（默认）或Ascend进行训练（单卡）
+ 单状态与多状态训练条件有两个不同：1）训练数据集不同；2）记忆层深度不同：单状态下记忆层深度为2层（num_memory_layers=2），多状态下记忆层深度为4层（num_memory_layers=4）

In [3]:
set_seed(0)
np.random.seed(0)

In [4]:
parser = argparse.ArgumentParser(description='eHDNN for Transonic buffet')
parser.add_argument("--mode", type=str, default="GRAPH", choices=["GRAPH", "PYNATIVE"],
                    help="Context mode, support 'GRAPH', 'PYNATIVE'")
parser.add_argument("--save_graphs", type=bool, default=False, choices=[True, False],
                    help="Whether to save intermediate compilation graphs")
parser.add_argument("--save_graphs_path", type=str, default="./summary")
parser.add_argument("--device_target", type=str, default="GPU", choices=["GPU", "Ascend"],
                    help="The target device to run, support 'Ascend', 'GPU'")
parser.add_argument("--device_id", type=int, default=0, help="ID of the target device")
parser.add_argument("--train_aoa_list", type=list, default=[35],
                    help="The type for training, [33 ,34 , 35 , 36 , 37 , 38] for multi_state training /n"
                         "[33],....,[38] for single_state training")
parser.add_argument("--num_memory_layers", type=int, default=2, choices=[2, 4],
                    help="The number of layers of the whole Memory layer， 2 in single_state and 4 in multi state")
parser.add_argument("--config_file_path", type=str, default="./config.yaml")
args = parser.parse_args(args=[])
context.set_context(mode=context.GRAPH_MODE if args.mode.upper().startswith("GRAPH") else context.PYNATIVE_MODE,
                    save_graphs=args.save_graphs, save_graphs_path=args.save_graphs_path,
                    device_target=args.device_target, device_id=args.device_id)
use_ascend = context.get_context(attr_key='device_target') == "Ascend"

## 训练超参数

In [5]:
config = load_yaml_config(args.config_file_path)
data_params = config["data"]
model_params = config["model"]
optimizer_params = config["optimizer"]

## 训练过程文件保存路径

In [6]:
summary_dir = optimizer_params["summary_dir"]
if not os.path.exists(summary_dir):
    os.mkdir(summary_dir)
ckpt_dir = os.path.join(summary_dir, 'ckpt')
if not os.path.exists(ckpt_dir):
    os.mkdir(ckpt_dir)

## 构建神经网络

In [7]:
model = ForwardNet(model_params["in_channels"],
                   model_params["out_channels"],
                   model_params["num_layers"],
                   args.num_memory_layers,
                   model_params["kernel_size_conv"],
                   model_params["kernel_size_lstm"])

## 优化器

In [8]:
loss_func = HybridLoss()
optimizer = nn.Adam(params=model.trainable_params(), learning_rate=optimizer_params["lr"])
if use_ascend:
    loss_scaler = DynamicLossScaler(1024, 2, 100)
    auto_mixed_precision(model, 'O1')
else:
    loss_scaler = None

## 训练框架

In [9]:
def forward_fn(x, y):
    pred = model(x)
    loss = loss_func(pred, y)
    if use_ascend:
        loss = loss_scaler.scale(loss)
    return loss

grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=False)
@jit
def train_step(x, y):
    loss, grads = grad_fn(x, y)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
        if all_finite(grads):
            grads = loss_scaler.unscale(grads)
    loss = ops.depend(loss, optimizer(grads))
    return loss
@jit
def eval_step(x, y):
    loss = forward_fn(x, y)
    if use_ascend:
        loss = loss_scaler.unscale(loss)
    return loss

## 数据集加载

In [10]:
print(f"======Load data sample ========")
data_dir = data_params["data_dir"]
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
url_1 = "https://download.mindspore.cn/mindscience/mindflow/dataset/applications/data_driven/airfoil/2D_unsteady/"
for aoa in args.train_aoa_list:
    url_2 = f"total{aoa}_puv.mat"
    url_aoa = urllib.parse.urljoin(url_1, url_2)
    urllib.request.urlretrieve(url_aoa, data_dir + '/' + url_2)
dataset_train, dataset_eval = create_dataset(data_dir,
                                             data_params["data_length"],
                                             data_params["train_ratio"],
                                             args.train_aoa_list)
print(f"==========End Load=============")

======Load data sample ========
input shape (2183, 16, 3, 200, 200)
label shape (2183, 1, 3, 200, 200)
==========End Load=============


## 数据下沉设置

In [11]:
train_sink_process = data_sink(train_step, dataset_train, sink_size=1)
eval_sink_process = data_sink(eval_step, dataset_eval, sink_size=1)
train_data_size, eval_data_size = dataset_train.get_dataset_size(), dataset_eval.get_dataset_size()

## 模型训练

In [12]:
for epoch in range(1, optimizer_params["epochs"] + 1):
    local_time_beg = time.time()
    model.set_train(True)
    epoch_train_loss = 0
    for _ in range(train_data_size):
        epoch_train_loss = ops.squeeze(train_sink_process(), axis=())
    train_loss.append(epoch_train_loss)
    print(f"epoch: {epoch} train loss: {epoch_train_loss} epoch time: {time.time() - local_time_beg:.2f}s")
    if epoch % optimizer_params["eval_interval"] == 0:
        print(f"=================Start Evaluation=====================")
        model.set_train(False)
        eval_loss = []
        for _ in range(eval_data_size):
            step_eval_loss = ops.squeeze(eval_sink_process(), axis=())
            eval_loss.append(step_eval_loss)
        epoch_eval_loss = sum(eval_loss) / len(eval_loss)
        print(f"epoch: {epoch} eval loss: {epoch_eval_loss}")
        print(f"==================End Evaluation======================")

    if epoch % optimizer_params["save_ckpt_interval"] == 0:
        save_checkpoint(model, f"{ckpt_dir}/net_{epoch}.ckpt")
    print(f"=====================End train========================")

pid:23104
========Start train============
epoch: 1 train loss: 1.3960425 epoch time: 329.46s
epoch: 2 train loss: 0.71045536 epoch time: 302.11s
epoch: 3 train loss: 0.63674355 epoch time: 294.79s
epoch: 4 train loss: 0.59894246 epoch time: 301.62s
epoch: 5 train loss: 0.5694136 epoch time: 311.56s
epoch: 6 train loss: 0.55112934 epoch time: 304.45s
epoch: 7 train loss: 0.5356926 epoch time: 310.83s
epoch: 8 train loss: 0.38834217 epoch time: 318.15s
epoch: 9 train loss: 0.31074354 epoch time: 320.24s
epoch: 10 train loss: 0.285696 epoch time: 306.86s
======Start Evaluation=========
epoch: 10 eval loss: 0.3148067
=======End Evaluation==========
=========End train=============


## 预测流场结果可视化

+ 运行prediction.py
+ 下图为基于多状态下训练完备的eHDNN模型实现对攻角3.75°（泛化状态）下非定常抖振流场单周期内的变化预测结果（展示压强场）

<figure class="harf">
    <img src="./images/375_pressure_cfd.gif" title="cfd" width="200"/>
    <img src="./images/375_pressure_prediction.gif" title="prediction" width="200"/>
    <img src="./images/375_pressure_abserror.gif" title="abs error" width="200"/>
</center>